In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random

import sys
sys.path.append('../input/tensorflow-great-barrier-reef')


In [ ]:
# utilities declaration
import shutil

from tqdm.notebook import tqdm # estimate and display the progress bar
tqdm.pandas() # to use progress.apply

from joblib import Parallel, delayed 
# parallel : readable parallel mapping
# delayed : 

In [ ]:
!pip install -qU wandb
!pip install -qU bbox-utility 

In [ ]:
from bbox.utils import coco2yolo, coco2voc, voc2yolo, draw_bboxes, load_image, clip_bbox, str2annot, annot2str

In [ ]:
np.random.seed(32)

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient # kaggle api
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
# constant declaration
IMAGE_SIZE = 2560
FOLD = 4
MODEL = 'yolov5s6'# yolov5, maybe using yolov5x yolov5l
BATCH = 4
EPOCHS = 30 # 40

PROJECT = 'great-barrier-reef'
NAME  = f'{MODEL}-dim{IMAGE_SIZE}-fold{FOLD}'

# REMOVE_NOBOX = True
ROOT_DIR = '../input/tensorflow-great-barrier-reef'
IMAGE_DIR = './images' # save images here for yolov5
LABEL_DIR = './labels' # save labels here for yolov5


In [ ]:
# make parent directory if it does not exist and pass a directory to a terminal
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [ ]:
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['old_image_path'] = f'{ROOT_DIR}/train_images/video_' + df.video_id.astype(str) + '/' + df.video_frame.astype(str) + '.jpg'
df['image_path'] = f'{IMAGE_DIR}/' + df.image_id + '.jpg'
df['label_path'] = f'{LABEL_DIR}/' + df.image_id + '.txt'
df['annotations'] = df['annotations'].progress_apply(eval) # eval for evaluates the specified expression, insecure
# still can't figure out what does this line do


In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x : len(x)) # count number of boxes in each image
data = (df.num_bbox > 0).value_counts() # 2 types of number of boxes
print(f'No bounding box : {data[0] * 100/(data[0] + data[1]):0.2f}% | With bounding box : {data[1] * 100/(data[0] + data[1]) : 0.2f}%')

remove no bouding box images

In [ ]:
len(df)

In [ ]:
df = df.drop(df.query('num_bbox == 0').sample(frac=.95).index)

In [ ]:
len(df)

In [ ]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x : len(x)) # count number of boxes in each image
data = (df.num_bbox > 0).value_counts() # 2 types of number of boxes
print(f'No bounding box : {data[0] * 100/(data[0] + data[1]):0.2f}% | With bounding box : {data[1] * 100/(data[0] + data[1]) : 0.2f}%')

In [ ]:
len(df)

enhancement functions

experiment image enhancement

In [ ]:
def recover_clahe(sceneRadiance) : # improvement of the above function 
    clahe = cv2.createCLAHE(clipLimit=7, tileGridSize=(14,14))
    for i in range(3) : 
        sceneRadiance[:,:,i] = clahe.apply((sceneRadiance[:,:,i]))
    return sceneRadiance


In [ ]:
def gamma_correction(img, gamma=1/0.6) : # gamma enhancement
    R = 255.0
    img = img.astype(np.uint32) / R
    new_image = R * np.power(img, gamma)
    return new_image.astype(np.uint8)

In [ ]:
def plot_img(img_dir,num_items,func,mode):
    img_list = random.sample(os.listdir(img_dir), num_items)

    for i in range(len(img_list)):
        full_path = img_dir + '/' + img_list[i]
        img_temp1 = plt.imread(full_path)
        img_temp_cv = cv2.imread(full_path)
        plt.figure(figsize=(20,15))
        plt.subplot(1,2,1)
        plt.imshow(img_temp1);
        plt.subplot(1,2,2)
        if mode == 'plt':
            plt.imshow(func(img_temp1));
        elif mode == 'cv2':
            plt.imshow(func(img_temp_cv));

In [ ]:
# vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
# num_items1 = 4
# plot_img(vid_0_dir,num_items1,recover_clahe,"cv2")

In [ ]:
# vid_0_dir = "../input/tensorflow-great-barrier-reef/train_images/video_0"
# num_items1 = 4
# plot_img(vid_0_dir,num_items1,gamma_correction,"cv2")

need to copy the images to working since YOLOv5 need to write and /kaggle/input does not allow to do it

In [ ]:
def plot_img2(img_dir,num_items,func, func2, mode):
    img_list = random.sample(os.listdir(img_dir), num_items)

    for i in range(len(img_list)):
        full_path = img_dir + '/' + img_list[i]
        img_temp1 = plt.imread(full_path)
        img_temp_cv = cv2.imread(full_path)
        plt.figure(figsize=(20,15))
        plt.subplot(1,2,1)
        plt.imshow(img_temp1);
        plt.subplot(1,2,2)
        if mode == 'plt':
            plt.imshow(func(img_temp1));
        elif mode == 'cv2':
            plt.imshow(func2(func(img_temp_cv)));

In [ ]:
# plot_img2(vid_0_dir,num_items1,gamma_correction, recover_clahe,  "cv2")

In [ ]:
def image_enhancement(image):
    return gamma_correction(recover_clahe(image))

In [ ]:
# def copy_files(row) : # copy after apply image enhancement 
#     # shutil.copyfile(row.old_image_path, row.image_path)
#     img = cv2.imread(row.old_image_path)
#     # img = image_enhancement(img)
#     cv2.imwrite(row.image_path, img)
#     return
def copy_files(row) : # copy after apply image enhancement 
    shutil.copyfile(row.old_image_path, row.image_path)
    return

make this progress faster by using joblib which uses parralel computing

In [ ]:
# image_paths = df.old_image_path.tolist() # why we use this
_ = Parallel(n_jobs = -1, backend='threading')( delayed(copy_files)(row) for _, row in tqdm(df.iterrows(), total=len(df)) )

# n_jobs = -1, using all the CPUs
# function tqdm to display these progress bar
# df.iterrows : DataFrame.iterrows is a generator which yields both the index and row (as a Series)
# delayed of joblib : to delay the execution of functions
## we'd like to call copy_files sometime later
## Returned is the tuple (function, [arguments  without keywords] , {argument with keywords})
## example : (delay, [row], {kwargs})

# delayed(copy_files)(row) for _, row in tqdm(df.iterrows(), total=len(df)) : return list and pass to the parallel function


In [ ]:
def get_bbox(annots) : 
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row) : 
    row['width'], row['height'] = imagesize.get(row['image_path']) # where is imagesize
    return row

colors = [(220,20,60)]

In [ ]:
df['bboxes'] = df.annotations.progress_apply(get_bbox)

In [ ]:
df['width'] = 1280
df['height'] = 720
df.head()

create labels files, convert from COCO format to YOLO format
must be normalized

In [ ]:
cnt = 0 
all_boxes = []
bboxes_info = []
for index in tqdm(range(df.shape[0])) : 
    row = df.iloc[index] # iloc only works with index but loc can and vice versa
    image_height = row.height
    image_width = row.width 
    # print(row.bboxes) each row.bboxes have several bboxes
    coco_bboxes = np.array(row.bboxes).astype(np.float32).copy() # convert to np array, change dtype 
    num_bbox = len(coco_bboxes) # num boxes in each image

    names = ['cots'] * num_bbox # copy these array to number of num_bbbox
    labels = np.array([0] * num_bbox)[..., None].astype(str) # name just a name, labels is marked from 0 to n 
    # [..., None] flatten these labels into a vector

    with open(row.label_path, 'w') as f: # open each files to write
        if num_bbox < 1 : 
            annot = '' # write nothing
            f.write(annot) 
            cnt += 1 # count the number of wrong filter 
            continue 
        # has bbox case 
        voc_bboxes = coco2voc(coco_bboxes, image_height, image_width) # convert coco to voc
        voc_bboxes = clip_bbox(voc_bboxes, image_height, image_width) # wtf 
        yolo_bboxes = voc2yolo(voc_bboxes, image_height, image_width).astype(str) # voc box to yolo format
        all_boxes.extend(yolo_bboxes.astype(float)) # list extend
        bboxes_info.extend([[row.image_id, row.video_id, row.sequence]] * len(yolo_bboxes)) # copy then 
        annots = np.concatenate([labels, yolo_bboxes], axis=1) # concatenate to the labels
        string = annot2str(annots) # convert it into string then
        f.write(string) # write it in the file

print('Missing :', cnt)

In [ ]:
from sklearn.model_selection import GroupKFold
kf = GroupKFold(n_splits = 5) # 5-fold with sepcific sequence is split to validation set
df = df.reset_index(drop = True)
df['fold'] = -1 # garbage value, will be fix later
# groups is df.sequence
for fold, (train_index, val_index) in enumerate(kf.split(df, y = df.video_id.tolist(), groups = df.sequence)) :
    df.loc[val_index, 'fold'] = fold
df.fold.value_counts()

In [ ]:
bbox_df = pd.DataFrame(np.concatenate([bboxes_info, all_boxes], axis = 1), 
                      columns = ['image_id', 'video_id', 'sequence', 'xmid', 'ymid', 'w', 'h'])
bbox_df[['xmid', 'ymid', 'w', 'h']] = bbox_df[['xmid', 'ymid', 'w', 'h']].astype(float)
bbox_df['area'] = bbox_df.w * bbox_df.h * 1280 * 720 # calculate the area
bbox_df = bbox_df.merge(df[['image_id','fold']], on='image_id', how='left')
bbox_df.head(2)

In [ ]:
from scipy.stats import gaussian_kde

all_boxes = np.array(all_boxes)

x_val = all_boxes[...,0]
y_val = all_boxes[...,1]

# Calculate the point density
xy = np.vstack([x_val,y_val])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (10, 10))
ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
plt.show()

In [ ]:
x_val = all_boxes[...,2]
y_val = all_boxes[...,3]

# Calculate the point density
xy = np.vstack([x_val,y_val])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots(figsize = (10, 10))
# ax.axis('off')
ax.scatter(x_val, y_val, c=z, s=100, cmap='viridis')
# ax.set_xlabel('bbox_width')
# ax.set_ylabel('bbox_height')
plt.show()

In [ ]:
import matplotlib as mpl
import seaborn as sns

f, ax = plt.subplots(figsize=(12, 6))
sns.despine(f)

sns.histplot(
    bbox_df,
    x="area", hue="fold",
    multiple="stack",
    palette="viridis",
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
)
ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
ax.set_xticks([500, 1000, 2000, 5000, 10000]);

In [ ]:
df2 = df[(df.num_bbox>0)].sample(100) # takes samples with bbox
y = 3; x = 2
plt.figure(figsize=(12.8*x, 7.2*y))
for idx in range(x*y):
    row = df2.iloc[idx]
    img           = load_image(row.image_path)
    image_height  = row.height
    image_width   = row.width
    with open(row.label_path) as f:
        annot = str2annot(f.read())
    bboxes_yolo = annot[...,1:]
    labels      = annot[..., 0].astype(int).tolist()
    names         = ['cots'] * len(bboxes_yolo)
    plt.subplot(y, x, idx+1)
    plt.imshow(draw_bboxes(img = img,
                           bboxes = bboxes_yolo, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = 'yolo',
                           line_thickness = 2))
    plt.axis('OFF')
plt.tight_layout()
plt.show()

In [ ]:
train_files = []
val_files = []
train_df = df.query('fold!=@FOLD') #wtf
valid_df = df.query('fold==@FOLD')
train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

In [ ]:
import yaml

cwd = '/kaggle/working/'

with open(os.path.join(cwd, 'train.txt'), 'w') as f: # write all train image directory in 
    for path in train_df.image_path.tolist() : 
        f.write(path + '\n')

with open(os.path.join(cwd, 'val.txt'), 'w') as f : 
    for path in valid_df.image_path.tolist() : 
        f.write(path + '\n')
        
data = dict(
    path = '/kaggle/working',
    train = os.path.join(cwd, 'train.txt'),
    val = os.path.join(cwd, 'val.txt'),
    nc = 1, # num classes
    name = ['cots']
)

with open(os.path.join(cwd, 'gbr.yaml'), 'w') as output_file : 
    yaml.dump(data, output_file, default_flow_style = False)
    
f = open(os.path.join(cwd, 'gbr.yaml'), 'r')
print('yaml:')
print(f.read())

In [ ]:
%%writefile /kaggle/working/hyp.yaml
lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.30  # image rotation (+/- deg)
translate: 0.10  # image translation (+/- fraction)
scale: 0.10  # image scale (+/- gain)
shear: 2.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 0.2  # image mosaic (probability)
mixup: 0.5 # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

In [ ]:
%cd /kaggle/working
!rm -r /kaggle/working/yolov5
! git clone https://github.com/ultralytics/yolov5 # clone
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

train yolov5

In [ ]:
!python train.py --img {IMAGE_SIZE} \
--batch {BATCH}\
--epochs {EPOCHS}\
--data /kaggle/working/gbr.yaml\
--hyp /kaggle/working/hyp.yaml\
--weights {MODEL}.pt\
--project {PROJECT} --name {NAME}\
--freeze 9 \
--exist-ok

In [ ]:
OUTPUT_DIR = '{}/{}'.format(PROJECT, NAME)
!ls {OUTPUT_DIR}

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch0.jpg'))

plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch1.jpg'))

plt.figure(figsize = (10, 10))
plt.imshow(plt.imread(f'{OUTPUT_DIR}/train_batch2.jpg'))

In [ ]:
fig, ax = plt.subplots(3, 2, figsize = (2*9,3*5), constrained_layout = True)
for row in range(3):
    ax[row][0].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg'))
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f'{OUTPUT_DIR}/val_batch{row}_labels.jpg', fontsize = 12)
    
    ax[row][1].imshow(plt.imread(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg'))
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f'{OUTPUT_DIR}/val_batch{row}_pred.jpg', fontsize = 12)
plt.show()

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/results.png'));

In [ ]:
plt.figure(figsize=(12,10))
plt.axis('off')
plt.imshow(plt.imread(f'{OUTPUT_DIR}/confusion_matrix.png'));

In [ ]:
for metric in ['F1', 'PR', 'P', 'R']:
    print(f'Metric: {metric}')
    plt.figure(figsize=(12,10))
    plt.axis('off')
    plt.imshow(plt.imread(f'{OUTPUT_DIR}/{metric}_curve.png'));
    plt.show()

- read the paper 
- load pretrain : done
- check the out-bound and not normalized box : done
- fine tune : done
- yolo5x out of memory => yolov5l : done
- 0-10% background image : done
- save model and get the submission from it : done
- large image size, done but not experiment yet
- little bit tune in number of freeze layer 


In [ ]:
os.chdir("/kaggle/")

In [ ]:
!ls
%cd working
!ls

In [ ]:
! rm -r images
! rm -r labels